In [1]:
from collections import defaultdict

from pathlib import Path
import os
import sys
import json
import pickle
#os.environ['CUDA_VISIBLE_DEVICES']='0'  # don't occupy GPU, inference can work on CPU
import torch
import logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from typing import List, Dict, Tuple

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
from sklearn.pipeline import Pipeline

from python_ev.dataset import load_parts, load_df
from vectorizers.bert_boris.bert_base_spwsi import BertBaseSPWSI
from subtask1_solutions.base_solution import Subtask1BaseVectorizerFactory
import flair

02/16/2019 15:42:18 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English
/home/vladimir/anaconda3/envs/dialogue2019gapping/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
from vectorizers.bert_boris.bert_flair_from_notebook_vectorizer import BertFlairFromNotebookVectorizer, test

vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

Loading bert pretrained: bert-base-multilingual-cased
Using torch device:  cuda
Loading bert done
Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _



In [3]:
sent = 'Ей он рассказывает одно, а нам - ток совершенно другое'.split(' ')
key = vectorizer.get_inst_id(7, 'dsdsg')
dct = {key: (sent, 7, '')}

In [4]:
vectorizer.predict_sent_substitute_representatives(dct)

Loading vocabulary from pretrained: bert-base-multilingual-cased


100%|██████████| 995526/995526 [00:01<00:00, 990391.87B/s]
generate_vecs: 0it [00:00, ?it/s]
Vectorization:   0%|          | 0/1 [00:00<?, ?it/s]
generate_vecs: 1it [00:00, 63.48it/s]00:00<00:00, 70.87it/s]
predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  6.72it/s]

Loading vocabulary done
INPUT TO BERT: [CLS] | Е | ##й | он | рассказывает | одно | , | а | нам | - | >>[MASK]<< | совершенно | другое | [SEP]


{'7.dsdsg': ['то и это - в он на во о что рассказывает есть ему , не говорит как только за его']}

In [44]:
def gen_list_of_inputs(sent, tok = 'MASK', lemma = ''):
    tokens = sent.split(' ')
    res = []
    for i in range(0, len(tokens) + 1):
        tokens = sent.split(' ')
        tokens.insert(i, tok)
        tokens.insert(i+1, tok) # убрать
        res.append({str(i):(tokens, i, lemma)})
    return res

In [45]:
#sent = 'Акциз на бензин в США при розничных продажах составляет 10 центов за литр, на дизельное топливо - 12 центов.'
#sent = 'Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению.'
#sent = 'Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.'
sent = 'По состоянию на 31 декабря 2010 года продажи алмазов сторонам, контролируемым государством, включали продажи Государственному фонду драгоценных металлов и драгоценных камней Российской Федерации в размере 25 млн. руб. (по состоянию на 31 декабря 2009 года - 27 060 млн. руб.).'
key = 'sent.sent'

bert_inps = gen_list_of_inputs(sent)

In [46]:
bert_inps

[{'0': (['MASK',
    'MASK',
    'По',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2010',
    'года',
    'продажи',
    'алмазов',
    'сторонам,',
    'контролируемым',
    'государством,',
    'включали',
    'продажи',
    'Государственному',
    'фонду',
    'драгоценных',
    'металлов',
    'и',
    'драгоценных',
    'камней',
    'Российской',
    'Федерации',
    'в',
    'размере',
    '25',
    'млн.',
    'руб.',
    '(по',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2009',
    'года',
    '-',
    '27',
    '060',
    'млн.',
    'руб.).'],
   0,
   '')},
 {'1': (['По',
    'MASK',
    'MASK',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2010',
    'года',
    'продажи',
    'алмазов',
    'сторонам,',
    'контролируемым',
    'государством,',
    'включали',
    'продажи',
    'Государственному',
    'фонду',
    'драгоценных',
    'металлов',
    'и',
    'драгоценных',
    'камней',
    'Российской',
    'Федерации',
    'в',
    

In [47]:
vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

results = []
for bert_inp in bert_inps[:1]:
    res = vectorizer.predict_sent_substitute_representatives(bert_inp)
    results.append((list(bert_inp.values())[0][0], list(res.values())[0]))

generate_vecs: 0it [00:00, ?it/s]
Vectorization:   0%|          | 0/1 [00:00<?, ?it/s]
generate_vecs: 1it [00:00, 34.29it/s]00:00<00:00, 36.63it/s]
predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  6.65it/s]

Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _

INPUT TO BERT: [CLS] | >>[MASK]<< | MA | ##S | ##K | По | состоянию | на | 31 | декабря | 2010 | года | про | ##даж | ##и | ал | ##ма | ##зов | сторона | ##м | , | кон | ##тро | ##ли | ##ру | ##ем | ##ым | государство | ##м | , | в | ##кл | ##ю | ##чали | про | ##даж | ##и | Г | ##ос | ##уда | ##рст | ##вен | ##ному | фонду | др | ##аг | ##о | ##цен | ##ных | металл | ##ов | и | др | ##аг | ##о | ##цен | ##ных | кам | ##ней | Российской | Федерации | в | размер | ##е | 25 | млн | . | р | ##уб | . | ( | по | состоянию | на | 31 | декабря | 2009 | года | - | 27 | 060 | млн | . | р | ##уб | . | ) | . | [SEP]


In [30]:
results

[(['MASK',
   'По',
   'состоянию',
   'на',
   '31',
   'декабря',
   '2010',
   'года',
   'продажи',
   'алмазов',
   'сторонам,',
   'контролируемым',
   'государством,',
   'включали',
   'продажи',
   'Государственному',
   'фонду',
   'драгоценных',
   'металлов',
   'и',
   'драгоценных',
   'камней',
   'Российской',
   'Федерации',
   'в',
   'размере',
   '25',
   'млн.',
   'руб.',
   '(по',
   'состоянию',
   'на',
   '31',
   'декабря',
   '2009',
   'года',
   '-',
   '27',
   '060',
   'млн.',
   'руб.).'],
  ['. - ( ) ; : , " * / » « 1 ] В К 2 ... [ 3']),
 (['По',
   'MASK',
   'состоянию',
   'на',
   '31',
   'декабря',
   '2010',
   'года',
   'продажи',
   'алмазов',
   'сторонам,',
   'контролируемым',
   'государством,',
   'включали',
   'продажи',
   'Государственному',
   'фонду',
   'драгоценных',
   'металлов',
   'и',
   'драгоценных',
   'камней',
   'Российской',
   'Федерации',
   'в',
   'размере',
   '25',
   'млн.',
   'руб.',
   '(по',
   'состоянию'

In [31]:
top_elems = 5

for elem in results: # ([elems], [str])
    fst = elem[0]
    snd = elem[1][0].split(' ')[:top_elems]
    snd = [elem for elem in snd if elem.isalpha()]
    if (not set(fst).isdisjoint(snd)):
        print(fst)
        print(snd)
        print('------------')

['По', 'MASK', 'состоянию', 'на', '31', 'декабря', '2010', 'года', 'продажи', 'алмазов', 'сторонам,', 'контролируемым', 'государством,', 'включали', 'продажи', 'Государственному', 'фонду', 'драгоценных', 'металлов', 'и', 'драгоценных', 'камней', 'Российской', 'Федерации', 'в', 'размере', '25', 'млн.', 'руб.', '(по', 'состоянию', 'на', '31', 'декабря', '2009', 'года', '-', '27', '060', 'млн.', 'руб.).']
['по', 'По', 'не']
------------
['По', 'состоянию', 'MASK', 'на', '31', 'декабря', '2010', 'года', 'продажи', 'алмазов', 'сторонам,', 'контролируемым', 'государством,', 'включали', 'продажи', 'Государственному', 'фонду', 'драгоценных', 'металлов', 'и', 'драгоценных', 'камней', 'Российской', 'Федерации', 'в', 'размере', '25', 'млн.', 'руб.', '(по', 'состоянию', 'на', '31', 'декабря', '2009', 'года', '-', '27', '060', 'млн.', 'руб.).']
['на', 'с']
------------
['По', 'состоянию', 'на', 'MASK', '31', 'декабря', '2010', 'года', 'продажи', 'алмазов', 'сторонам,', 'контролируемым', 'государств

In [34]:
vectorizer.tokenizer.tokenize('элементарных')

['элемент', '##ар', '##ных']

In [41]:
vectorizer.predict_sent_substitute_representatives(bert_inp)

generate_vecs: 1it [00:00, 79.34it/s]
predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  6.79it/s]


{'40': ['. В - По К С ( , На А Г Р Про М П У по О ) :']}

In [48]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [153]:
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [90]:
text = "Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению."

In [91]:
tok = tokenizer.tokenize(text)

In [94]:
tok

['я',
 'ду',
 '##ма',
 '##ю',
 ',',
 'она',
 'вскоре',
 'будет',
 'на',
 'но',
 '##гах',
 'и',
 'через',
 'несколько',
 'д',
 '##не',
 '##и',
 '[MASK]',
 'на',
 'пути',
 'к',
 'вы',
 '##здо',
 '##ров',
 '##лению',
 '.']

In [93]:
tok.insert(17, '[MASK]')

In [68]:
tok.insert(0, '[MASK]')

In [133]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tok)

In [134]:
segments_ids = [0 for x in range(len(indexed_tokens))]

In [137]:
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
segments_tensors = torch.tensor([segments_ids]).to('cuda')
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [150]:
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')

In [254]:
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

In [99]:
predicted_index = torch.argmax(predictions[0, 17]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

In [132]:
predicted_token

22366

In [377]:
def bottom_layer(sent, verb_list, tokenizer, index, model, threshold):
    # получаем предложение, список глаголов, токенизатор, индекс конца слова; возвращаем "похожие" токенизированные слова
    # также подаём на вход модель; threshold - количество получаемых элементов
    model.to('cuda')
    tok_sent = tokenizer.tokenize(sent)
    if (len(verb_list) == 1):
        verb = verb_list[0] # список из k элементов (k - количество токенов)
        num_tokens = len(verb)
        for i in range(index+2, len(tok_sent)+1):
            if (i == len(tok_sent) or tok_sent[i].isalpha() or tok_sent[i].isdigit()):
                new_tok = tok_sent.copy()
                print(tok_sent[i-1])
                print('=====')
                # тогда перед этим всем вставляем num_tokens токенов
                for _ in range(num_tokens):
                    new_tok.insert(i, '[MASK]')
                indexed_tokens = torch.tensor([tokenizer.convert_tokens_to_ids(new_tok)]).to('cuda')
                segment_ids = torch.tensor([[0 for x in range(indexed_tokens.shape[1])]]).to('cuda')
                #print(indexed_tokens)
                #print(segment_ids)
                with torch.no_grad():
                    predictions = model(indexed_tokens, segment_ids)
                predictions = predictions.cpu()
                for j in range(i, i+num_tokens):
                    #predictions.cpu()[0, j].numpy().argsort(axis = 1).T[-10:][::-1].T
                    predicted_indexes = predictions.cpu()[0, j].numpy().argsort()[-threshold:][::-1]
                    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes)
                    print(predicted_tokens)
                print('--------------')

In [397]:
st = 'В любой момент времени в работе ( в передаче момента) находится восемь цевочных осей (четыре зацепляются с корпусом, четыре с ведомым валом).'
tokenizer.tokenize(st)
#tokenizer.tokenize('содержат')

['в',
 'л',
 '##ю',
 '##бо',
 '##и',
 'момент',
 'времени',
 'в',
 'работе',
 '(',
 'в',
 'передач',
 '##е',
 'момента',
 ')',
 'находится',
 'восемь',
 'це',
 '##во',
 '##чных',
 'ос',
 '##еи',
 '(',
 'четыре',
 'за',
 '##це',
 '##п',
 '##ляются',
 'с',
 'корпус',
 '##ом',
 ',',
 'четыре',
 'с',
 'в',
 '##ед',
 '##омы',
 '##м',
 'ва',
 '##лом',
 ')',
 '.']

In [398]:
bottom_layer(st, [['за', '##це', '##п', '##ляются']], tokenizer, 0, model, 10)

##и
=====
['ос', 'в', 'с', 'це', 'раз', 'вы', 'при', 'же', 'про', 'у']
['##ми', '.', '##м', 'в', '##ом', '##ю', '##й', '##и', ',', 'на']
['.', 'в', '##ми', 'на', '##м', '##ом', ',', 'за', '-', '##ю']
['в', 'на', 'каждый', '##ный', 'за', 'любой', 'В', 'На', '.', 'данный']
--------------
момент
=====
['##ами', '##ы', '##ом', '##и', '##ными', '##ов', '##ой', '##ми', '##ями', '##ками']
['.', 'в', 'и', ':', 'по', 'на', ',', 'за', ';', 'при']
['зависимости', '.', 'в', 'по', 'В', 'течение', 'на', 'за', 'Для', 'каждом']
['##го', 'этого', 'каждого', 'основного', 'момент', '##м', 'от', 'того', 'же', 'всего']
--------------
времени
=====
['.', 'в', '(', 'и', ':', ',', '-', 'на', 'за', 'не']
['в', '.', 'на', 'за', 'с', '-', 'один', 'по', 'время', 'секунд']
['.', ',', ')', 'время', 'момент', 'времени', 'минут', ':', '##е', 'секунд']
['.', ';', ',', ':', 'время', 'момент', 'и', 'что', 'времени', ')']
--------------
в
=====
['течение', 'работе', 'за', 'про', 'об', 'каждой', 'каждом', 'па', 'этом', 'т

In [218]:
tokenizer.tokenize('дифференциальной')

['ди', '##ф', '##фер', '##ен', '##циал', '##ьно', '##и']

In [261]:
predictions.cpu()[0]

torch.Size([26, 119547])

In [358]:
tmp = predictions.cpu()[0, 1].numpy().argsort()[-10:][::-1]

In [371]:
tokenizer.convert_ids_to_tokens(tmp)

['ду', '.', ',', 'la', 'des', ')', 'di', 'il', '。', 'le']

In [270]:
tmp.sort(axis = 1)

In [274]:
tmp[0]

array([1, 2])